# Ceneo Scraper

# Imports

In [6]:
import requests
from bs4 import BeautifulSoup

Send request to Ceneo.pl serivce

In [4]:
product_id = "108290707"
url = f"https://www.ceneo.pl/{product_id}"
response = requests.get(url)
response.status_code

200

Convert plain text HTML code into DOM structure

In [12]:
page_dom = BeautifulSoup(response.text, 'html.parser')
print(type(page_dom))
opinions = page_dom.select('div.js_product-reviews')
print(type(opinions))
opinion = page_dom.select_one('div.js_product-reviews')
print(opinion)

<class 'bs4.BeautifulSoup'>
<class 'bs4.element.ResultSet'>
